In [1]:
import os
import gc
import json
import time
import joblib
import warnings
import traceback
import numpy as np
import pandas as pd
import optuna
import pandas_ta as ta
import tensorflow as tf
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.signal import argrelextrema
from numba import jit
from collections import Counter
import numpy as np
import pandas as pd
import optuna
import joblib
import json
import gc
import traceback
import os
from datetime import datetime
import tensorflow as tf
# Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
# Numba (For Fast Backtesting)
from numba import jit

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print(f"🚀 GPU Detected: {tf.config.list_physical_devices('GPU')}")
else:
    print("⚠️ No GPU detected. Using CPU.")
    

def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    # 1. 로그 수익률 생성
    if 'eth_log_return' not in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['eth_log_return'] = np.log(df_transformed[close_col] / df_transformed[close_col].shift(1))
    
    # 2. ATR 변환 (달러 -> 비율)
    if 'ATR_84' in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['ATR_84'] = (df_transformed['ATR_84'] / df_transformed[close_col]) * 100
            df_transformed['ATR_84'] = df_transformed['ATR_84'].replace([np.inf, -np.inf], 0).fillna(0)

    remove_patterns = [
        '_Close', '_Open', '_High', '_Low', '_Volume', 
        'tvl', 'mcap', 'sp500', 'gold', 'dxy', 'vwap' 
    ]
    
    keep_keywords = [
        '_pct', '_ratio', '_chg', '_diff', '_dist', '_score', 'zscore', 
        'natr', 'rsi', 'mfi', 'adx', 'cci', 'vr', 'shadow', 
        'div', 'spike', 'climax', 'absorption', 'breakout', 'price_vs',
        'ATR_84'
    ]
    
    cols_to_remove = []
    for col in df_transformed.columns:
        if col in ['date', 'target', 'next_direction', 'next_close', 'next_log_return', 'take_profit_price', 'stop_loss_price', 'real_entry_price', 'ATR_84']:
            continue
            
        is_dangerous = any(p in col.lower() for p in remove_patterns) or col in ['EMA_72', 'SMA_120', 'OBV']
        is_safe = any(k in col.lower() for k in keep_keywords)
        
        if is_dangerous and not is_safe:
            cols_to_remove.append(col)

    if cols_to_remove:
        df_transformed.drop(cols_to_remove, axis=1, inplace=True)
        
    return df_transformed



2025-11-29 08:25:18.414914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 08:25:18.414955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 08:25:18.416335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 08:25:18.423837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 08:25:19.186236: W tensorflow/comp

🚀 GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
@jit(nopython=True)
def compute_targets_with_hourly_numba_4h(
    base_dates_ts, base_atr, hourly_dates_ts, hourly_open,
    hourly_high, hourly_low, lookahead_periods, profit_mult, stop_mult
):
    n = len(base_dates_ts)
    # 초기화: 기본값은 -1 (유효하지 않음) 또는 NaN
    targets = np.full(n, -1, dtype=np.int32)
    entry_prices = np.full(n, np.nan, dtype=np.float64) # 명시적 float64
    tp_prices = np.full(n, np.nan, dtype=np.float64)
    sl_prices = np.full(n, np.nan, dtype=np.float64)
    
    four_hour_ms = 14400000
    lookahead_ms = lookahead_periods * four_hour_ms
    MIN_PROFIT_THRESHOLD = 0.0025 
    
    h_start = 0
    
    # 마지막 데이터는 lookahead 계산 불가하므로 n-1까지만 루프
    for i in range(n - 1):
        atr = base_atr[i]
        # ATR이 없거나 0이면 계산 불가 -> continue (초기값 -1/NaN 유지)
        if np.isnan(atr) or atr <= 0: 
            continue
        
        entry_start_ts = base_dates_ts[i] + four_hour_ms
        entry_end_ts = entry_start_ts + lookahead_ms
        
        first_entry_idx = -1
        
        # 1시간봉 매칭 (h_start부터 검색하여 속도 최적화)
        for h in range(h_start, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_start_ts:
                first_entry_idx = h
                h_start = h # 다음 루프를 위해 시작점 업데이트
                break
        
        # 매칭되는 1시간봉이 없으면(데이터 끝부분 등) 계산 불가 -> continue
        if first_entry_idx == -1:
            continue
        
        # 진입 가격 확정
        entry_price = hourly_open[first_entry_idx]
        tp = entry_price + (atr * profit_mult)
        sl = entry_price - (atr * stop_mult)
        
        # [중요] 배열에 값 할당 (i 인덱스 위치에 정확히!)
        entry_prices[i] = entry_price
        tp_prices[i] = tp
        sl_prices[i] = sl
        
        result = 0 # 기본적으로 실패(0)로 시작
        final_idx = -1 
        
        for h in range(first_entry_idx, len(hourly_dates_ts)):
            # 1. 시간 초과 (Lookahead 기간 종료)
            if hourly_dates_ts[h] >= entry_end_ts:
                final_idx = h
                break
            
            # 2. 손절 (Stop Loss) - 최우선 확인
            if hourly_low[h] <= sl:
                result = 0
                final_idx = -1 # 손절 당했으므로 Timeout 수익 체크 불필요
                break
                
            # 3. 익절 (Take Profit)
            if hourly_high[h] >= tp:
                result = 1
                final_idx = -1 # 익절 했으므로 종료
                break
        
        # 4. Timeout 처리 (아직 포지션 보유 중일 때 수익률 체크)
        if final_idx != -1:
            exit_price = hourly_open[final_idx]
            return_rate = (exit_price - entry_price) / entry_price
            
            if return_rate >= MIN_PROFIT_THRESHOLD:
                result = 1
            else:
                result = 0

        # 결과 할당
        targets[i] = result
    
    return targets, entry_prices, tp_prices, sl_prices


def create_targets_4h(df_4h, df_1h, lookahead=30, profit_mult=1.5, stop_mult=1.0, **kwargs):
    df_target = df_4h.copy()
    hourly_df = df_1h.copy()
    
    df_target['date'] = pd.to_datetime(df_target['date'])
    hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
    
    hourly_df.columns = hourly_df.columns.str.lower()
    
    if 'ATR_84' not in df_target.columns:
        df_target['ATR_84'] = ta.atr(
            df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=84
        )
    
    df_target = df_target.sort_values('date').reset_index(drop=True)
    hourly_df = hourly_df.sort_values('datetime').reset_index(drop=True)
    
    base_dates_ts = df_target['date'].astype(np.int64).values // 10**6
    base_atr = df_target['ATR_84'].fillna(method='ffill').fillna(0).to_numpy()
    
    hourly_dates_ts = hourly_df['datetime'].astype(np.int64).values // 10**6
    hourly_open = hourly_df['open'].astype(np.float64).to_numpy()
    hourly_high = hourly_df['high'].astype(np.float64).to_numpy()
    hourly_low = hourly_df['low'].astype(np.float64).to_numpy()
    
    targets, entry_prices, tp_prices, sl_prices = compute_targets_with_hourly_numba_4h(
        base_dates_ts, base_atr, hourly_dates_ts, hourly_open, hourly_high, hourly_low,
        lookahead, profit_mult, stop_mult
    )
    
    df_target['next_direction'] = targets
    df_target['real_entry_price'] = entry_prices
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    
    if lookahead > 0:
        df_target = df_target.iloc[:-lookahead]
    
    valid_before = len(df_target)
    df_target = df_target[df_target['next_direction'] != -1].reset_index(drop=True)
    valid_after = len(df_target)
    
    print(f"Valid Samples: {valid_after}/{valid_before} (Removed: {valid_before - valid_after})")
    
    return df_target


In [3]:
def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    if len(X_train) > 10000:
        idx = np.random.choice(len(X_train), 10000, replace=False)
        X_sub = X_train.iloc[idx]
        y_sub = y_train.iloc[idx]
    else:
        X_sub, y_sub = X_train, y_train

    if task == 'class':
        mi_scores = mutual_info_classif(X_sub, y_sub, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_sub, y_sub, random_state=42, n_neighbors=3)
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_sub, y_sub)
    rfe_features = X_train.columns[rfe.support_].tolist()

    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_sub, y_sub)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}

def select_features_multi_target(X_train, y_train, target_type='direction', top_n=20):
    atr_col_name = 'ATR_84'
    if target_type == 'direction':
        selected, stats = select_features_verified(X_train, y_train['next_direction'], task='class', top_n=top_n)
        
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            if len(selected) > 0: selected.pop()
            selected.insert(0, atr_col_name)
            
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats

def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, atr_col_name='ATR_84'): 
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min()} ~ {train_df['date'].max()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min()} ~ {val_df['date'].max()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min()} ~ {test_df['date'].max()} (N={len(test_df)})")

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_84', 'real_entry_price' 
    ]

    train_processed = train_df.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_df.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_df.dropna(subset=target_cols).reset_index(drop=True)

    exclude_cols = [col for col in target_cols if col != atr_col_name] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_sel), columns=selected_features)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val_sel), columns=selected_features)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_sel), columns=selected_features)

    return {
        'train': {'X': X_train_scaled, 'y': y_train.reset_index(drop=True), 'dates': train_processed['date'].reset_index(drop=True)},
        'val': {'X': X_val_scaled, 'y': y_val.reset_index(drop=True), 'dates': val_processed['date'].reset_index(drop=True)},
        'test': {'X': X_test_scaled, 'y': y_test.reset_index(drop=True), 'dates': test_processed['date'].reset_index(drop=True)},
        'scaler': scaler, 
        'selected_features': selected_features,
        'stats': {
            'fold_idx': fold_idx if fold_idx is not None else split_data.get('fold_idx', 0),
            'fold_type': split_data.get('fold_type', 'unknown')
        }
    }

def split_walk_forward_method(df, train_start_date, final_test_start='2025-01-01', 
                              initial_train_size=800, val_size=150, test_size=150, 
                              step=150, gap_size=30):
    
    df_period = df[df['date'] >= pd.to_datetime(train_start_date)].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    total_len = len(df_period)
    folds = []
    current_test_end = total_len
    
    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0: break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })
        current_test_end = test_start_idx - gap_size
    
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        if len(pre_final_df) >= (initial_train_size + val_size + gap_size):
            final_val_end_idx = len(pre_final_df)
            final_val_start_idx = final_val_end_idx - val_size
            final_train_end_idx = final_val_start_idx - gap_size
            final_train_start_idx = final_train_end_idx - initial_train_size
            
            final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
            final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
            
            folds.append({
                'train': final_train_data,
                'val': final_val_data,
                'test': final_test_df,
                'fold_idx': len(folds) + 1,
                'fold_type': 'final_holdout'
            })
            
    return folds

def build_complete_pipeline_corrected(df_raw, df_hour, train_start_date, **kwargs): 
    print(f"\n Pipeline Started... (Train Start: {train_start_date})")


    lookahead = kwargs.get('lookahead_periods', 30) 
    profit_mult = kwargs.get('profit_mult', 2.0)
    stop_mult = kwargs.get('stop_mult', 1.0)

    df = create_targets_4h(df_raw, df_hour,
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult
    )

    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    print(f"Final Data Shape: {df.shape}")
    
    initial_train_size = kwargs.get('initial_train_days', 800) * 6
    val_size = 150 * 6
    test_size = 150 * 6
    gap_size = lookahead

    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=initial_train_size,
        val_size=val_size,
        test_size=test_size,
        step=150 * 6,
        gap_size=gap_size
    )
    print(f"Data Split Completed. Total {len(splits)} folds generated.")

    result = []
    for fold in splits:
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx'],
            atr_col_name='ATR_84'
        )
        result.append(res)

    return result

In [4]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import gc
import traceback
import os
import joblib
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

class DirectionModels:
    
    @staticmethod
    def get_class_weights(y_train):
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        return dict(zip(classes, weights))

    @staticmethod
    def _calculate_expectancy_percentile(y_true, y_prob, rr_ratio=1.5, min_trades=20):
        """
        [수정됨] 고정 임계값 대신 Percentile 기반으로 최적의 Expectancy 계산
        """
        best_exp = -999.0
        best_th = 0.5
        best_metrics = {'win_rate': 0.0, 'trades': 0}
        
        # 상위 1% ~ 50% 사이를 탐색 (너무 널널하면 의미 없음)
        percentiles = np.arange(50, 99, 2) 
        
        for pct in percentiles:
            th = np.percentile(y_prob, pct)
            if th < 0.5: continue # 최소 확률 조건
            
            preds = (y_prob >= th).astype(int)
            n_trades = preds.sum()
            
            if n_trades < min_trades: continue
            
            wins = ((preds == 1) & (y_true == 1)).sum()
            losses = n_trades - wins
            
            win_rate = wins / n_trades
            exp = (wins * rr_ratio - losses * 1.0) / n_trades
            
            # 승률 페널티 (안정성 확보)
            if win_rate < 0.4: exp -= 0.5
            
            # 거래 횟수 가산점 (신뢰도 확보)
            score = exp * np.log1p(n_trades)
            
            if score > best_exp: # Score 기준으로 갱신하되, 저장은 원본 Exp로
                best_exp = score
                best_th = th
                best_metrics = {'win_rate': win_rate, 'trades': n_trades, 'real_exp': exp}
                
        # 반환값: 최적화용 점수(Score), 임계값, 메타데이터(실제 Exp 포함)
        return best_exp, best_th, best_metrics

    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val, rr_ratio=1.5):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 300, 800), # 과적합 방지 위해 상한 축소
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_samples_split': trial.suggest_int('min_samples_split', 50, 200),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 50),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'class_weight': 'balanced',
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = RandomForestClassifier(**params)
            model.fit(X_train, y_train)
            
            val_prob = model.predict_proba(X_val)[:, 1]
            # [수정] Percentile 기반 평가 함수 호출
            score, _, _ = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
            return score

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=15) # Trial 횟수 조절
        
        best_model = RandomForestClassifier(**study.best_params, class_weight='balanced', random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        # 로깅용
        train_prob = best_model.predict_proba(X_train)[:, 1]
        val_prob = best_model.predict_proba(X_val)[:, 1]
        
        _, _, train_meta = DirectionModels._calculate_expectancy_percentile(y_train, train_prob, rr_ratio=rr_ratio)
        _, _, val_meta = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
        
        # 메타데이터에서 실제 Exp 꺼내기 (없으면 0)
        tr_exp = train_meta.get('real_exp', 0.0)
        val_exp = val_meta.get('real_exp', 0.0)

        print(f"  [RandomForest] Train Exp: {tr_exp:.4f}R | Val Exp: {val_exp:.4f}R (Trades: {val_meta['trades']})")
        
        return best_model

    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val, rr_ratio=1.5):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 60),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'min_child_samples': trial.suggest_int('min_child_samples', 50, 300), # 과적합 방지 상향
                'subsample': trial.suggest_float('subsample', 0.6, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
                'objective': 'binary',
                'metric': 'binary_logloss',
                'class_weight': 'balanced',
                'verbosity': -1,
                'n_jobs': -1,
                'random_state': 42
            }
            
            model = lgb.LGBMClassifier(**params)
            # Early Stopping을 위한 콜백
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)]
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
            
            val_prob = model.predict_proba(X_val)[:, 1]
            score, _, _ = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
            return score

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 'objective': 'binary', 'metric': 'binary_logloss', 
            'class_weight': 'balanced', 'verbosity': -1, 'n_jobs': -1, 'random_state': 42
        })
        
        final_model = lgb.LGBMClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        _, _, train_meta = DirectionModels._calculate_expectancy_percentile(y_train, train_prob, rr_ratio=rr_ratio)
        _, _, val_meta = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
        
        tr_exp = train_meta.get('real_exp', 0.0)
        val_exp = val_meta.get('real_exp', 0.0)
        
        print(f"  [LightGBM]     Train Exp: {tr_exp:.4f}R | Val Exp: {val_exp:.4f}R (Trades: {val_meta['trades']})")
        
        return final_model

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val, rr_ratio=1.5):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        neg, pos = np.bincount(y_train.astype(int))
        scale_pos_weight = neg / pos

        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'min_child_weight': trial.suggest_int('min_child_weight', 10, 50), # 과적합 방지 상향
                'subsample': trial.suggest_float('subsample', 0.6, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
                'gamma': trial.suggest_float('gamma', 1.0, 5.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
                'scale_pos_weight': scale_pos_weight,
                'objective': 'binary:logistic',
                'eval_metric': 'logloss',
                'tree_method': 'hist',
                'early_stopping_rounds': 30,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            
            val_prob = model.predict_proba(X_val)[:, 1]
            score, _, _ = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
            return score

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 
            'tree_method': 'hist', 'scale_pos_weight': scale_pos_weight,
            'early_stopping_rounds': 50, 'random_state': 42, 'n_jobs': -1
        })
        
        final_model = xgb.XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        _, _, train_meta = DirectionModels._calculate_expectancy_percentile(y_train, train_prob, rr_ratio=rr_ratio)
        _, _, val_meta = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
        
        tr_exp = train_meta.get('real_exp', 0.0)
        val_exp = val_meta.get('real_exp', 0.0)
        
        print(f"  [XGBoost]      Train Exp: {tr_exp:.4f}R | Val Exp: {val_exp:.4f}R (Trades: {val_meta['trades']})")
        
        return final_model

    @staticmethod
    def catboost(X_train, y_train, X_val, y_val, rr_ratio=1.5):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'iterations': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'depth': trial.suggest_int('depth', 4, 8),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 30), # 정규화 강화
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
                'loss_function': 'Logloss',
                'eval_metric': 'Logloss',
                'auto_class_weights': 'Balanced',
                'logging_level': 'Silent',
                'random_seed': 42,
                'od_type': 'Iter',
                'od_wait': 30,
                'allow_writing_files': False
            }
            
            model = cb.CatBoostClassifier(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
            
            val_prob = model.predict_proba(X_val)[:, 1]
            score, _, _ = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
            return score

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'iterations': 1000, 'loss_function': 'Logloss', 'eval_metric': 'Logloss',
            'auto_class_weights': 'Balanced', 'logging_level': 'Silent',
            'random_seed': 42, 'od_type': 'Iter', 'od_wait': 50, 'allow_writing_files': False
        })
        
        final_model = cb.CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        _, _, train_meta = DirectionModels._calculate_expectancy_percentile(y_train, train_prob, rr_ratio=rr_ratio)
        _, _, val_meta = DirectionModels._calculate_expectancy_percentile(y_val, val_prob, rr_ratio=rr_ratio)
        
        tr_exp = train_meta.get('real_exp', 0.0)
        val_exp = val_meta.get('real_exp', 0.0)
        
        print(f"  [CatBoost]     Train Exp: {tr_exp:.4f}R | Val Exp: {val_exp:.4f}R (Trades: {val_meta['trades']})")
        
        return final_model

ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.best_thresholds = {}
        self.save_models = save_models
        self.models = {} if save_models else None
        self.prediction_logs = {}
        
    def optimize_threshold(self, y_true, buy_prob, min_trades=20, reward_risk_ratio=1.5):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            pct = trial.suggest_float('percentile', 50, 99)
            th = np.percentile(buy_prob, pct)
            
            if th < 0.5: return -999.0
            
            preds = (buy_prob >= th).astype(int)
            n_trades = np.sum(preds == 1)
            
            if n_trades < min_trades: return -10.0
            
            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            
            win_rate = wins / n_trades
            expectancy = ((wins * reward_risk_ratio) - (losses * 1.0)) / n_trades
            
            if win_rate < 0.4: expectancy -= 0.5
            
            score = expectancy * np.log1p(n_trades)
            return score

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_pct = study.best_params['percentile']
        best_th = np.percentile(buy_prob, best_pct)
        
        return max(best_th, 0.5)

    def evaluate_model(self, model, X_train, y_train, X_val, y_val, X_test, y_test, 
                       model_name, profit_mult=1.5, stop_mult=1.0,
                       test_dates=None, test_prices=None, y_test_df=None):  
        
        def calculate_all_metrics(y_true, probs, threshold, prefix):
            preds = (probs >= threshold).astype(int)
            n_trades = np.sum(preds == 1)

            acc = accuracy_score(y_true, preds)
            prec = precision_score(y_true, preds, zero_division=0)
            rec = recall_score(y_true, preds, zero_division=0)
            f1 = f1_score(y_true, preds, zero_division=0)
            try: auc = roc_auc_score(y_true, probs)
            except: auc = 0.5

            if n_trades > 0:
                wins = np.sum((preds == 1) & (y_true == 1))
                losses = n_trades - wins
                win_rate = wins / n_trades
                expectancy = ((wins * rr_ratio) - (losses * 1.0)) / n_trades
            else:
                win_rate, expectancy = 0.0, 0.0

            return {
                f"{prefix}_Acc": acc, f"{prefix}_Prec": prec, f"{prefix}_Rec": rec,
                f"{prefix}_F1": f1, f"{prefix}_AUC": auc,
                f"{prefix}_Exp": expectancy, f"{prefix}_WinRate": win_rate, f"{prefix}_Trades": n_trades
                }
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        
        tr_prob = model.predict_proba(X_train)[:, 1]
        val_prob = model.predict_proba(X_val)[:, 1]
        te_prob = model.predict_proba(X_test)[:, 1]
        
        min_trades_val = max(20, int(len(y_val) * 0.02))
        
        best_th = self.optimize_threshold(y_val.astype(int), val_prob, min_trades=min_trades_val, reward_risk_ratio=rr_ratio)
        self.best_thresholds[model_name] = best_th

        tr_pos_ratio = np.mean(y_train)
        val_pos_ratio = np.mean(y_val)
        te_pos_ratio = np.mean(y_test)
        
        metrics_train = calculate_all_metrics(y_train.astype(int), tr_prob, best_th, "Train")
        metrics_val = calculate_all_metrics(y_val.astype(int), val_prob, best_th, "Val")
        metrics_test = calculate_all_metrics(y_test.astype(int), te_prob, best_th, "Test")
        
        result = {
            'Model': model_name,
            'Threshold': best_th,
            'Train_PosRatio': tr_pos_ratio, 
            'Val_PosRatio': val_pos_ratio, 
            'Test_PosRatio': te_pos_ratio,
            **metrics_train,
            **metrics_val,
            **metrics_test
        }
        
        print(f"    -> [EVAL] {model_name:<12} | Te_Exp: {result['Test_Exp']:.3f}R | Te_Trades: {result['Test_Trades']} | Th: {best_th:.4f}")

        if test_dates is not None:
            close_vals = test_prices['ETH_Close'].values if isinstance(test_prices, pd.DataFrame) else test_prices
            open_vals = test_prices['ETH_Open'].values if isinstance(test_prices, pd.DataFrame) else np.zeros(len(te_prob))
            high_vals = test_prices['ETH_High'].values if isinstance(test_prices, pd.DataFrame) else np.zeros(len(te_prob))
            low_vals = test_prices['ETH_Low'].values if isinstance(test_prices, pd.DataFrame) else np.zeros(len(te_prob))

            tp_vals = y_test_df['take_profit_price'].values if y_test_df is not None else np.zeros(len(te_prob))
            sl_vals = y_test_df['stop_loss_price'].values if y_test_df is not None else np.zeros(len(te_prob))
            entry_vals = y_test_df['real_entry_price'].values if y_test_df is not None else np.zeros(len(te_prob))

            pred_df = pd.DataFrame({
                'timestamp': test_dates,
                'open': open_vals, 'high': high_vals, 'low': low_vals, 'close': close_vals,
                'prob': te_prob, 'threshold': best_th, 'signal': (te_prob >= best_th).astype(int),
                'actual_target': y_test.astype(int),
                'take_profit_price': tp_vals, 'stop_loss_price': sl_vals, 'real_entry_price': entry_vals
            })
            if not isinstance(pred_df.index, pd.DatetimeIndex):
                pred_df.set_index('timestamp', inplace=True)
            self.prediction_logs[model_name] = pred_df

        self.results.append(result)
        if self.save_models: self.models[model_name] = model
        
        return result
    
    def get_summary_dataframe(self): return pd.DataFrame(self.results)
    def get_models_dict(self): return self.models or {}
    def get_prediction_logs(self): return self.prediction_logs
    def get_best_thresholds(self): return self.best_thresholds

class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    def _prepare_target(self, y_data):
        # Extracts the first column (target) and flattens it to 1D array
        if isinstance(y_data, pd.DataFrame):
            y_data = y_data.iloc[:, 0].values
        elif isinstance(y_data, pd.Series):
            y_data = y_data.values
        y_data = np.array(y_data).flatten()
        y_data = np.nan_to_num(y_data, nan=0.0)
        return np.round(y_data).astype(int)

    def train_all_models(self, X_train, y_train, X_val, y_val, X_test, y_test, 
                         profit_mult, stop_mult, ml_models,
                         test_dates=None, test_prices=None):
        
        # 1. Prepare numeric arrays for training
        y_train_arr = self._prepare_target(y_train)
        y_val_arr = self._prepare_target(y_val)
        y_test_arr = self._prepare_target(y_test)
        
        real_rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        print(f"  [Info] Training with RR_Ratio: {real_rr_ratio:.2f}") 
        
        for config in ml_models:
            try:
                # Train Model
                if config.get('needs_val', False):
                    model = config['func'](X_train, y_train_arr, X_val, y_val_arr, rr_ratio=real_rr_ratio)
                else:
                    model = config['func'](X_train, y_train_arr)
                
                # Evaluate Model
                self.evaluator.evaluate_model(
                    model=model, 
                    X_train=X_train, 
                    y_train=y_train_arr, 
                    X_val=X_val, 
                    y_val=y_val_arr, 
                    X_test=X_test, 
                    y_test=y_test_arr,         
                    model_name=config['name'], 
                    profit_mult=profit_mult, 
                    stop_mult=stop_mult,
                    test_dates=test_dates, 
                    test_prices=test_prices,
                    y_test_df=y_test         
                )
                del model
                gc.collect()
            except Exception as e:
                print(f"[Error] Failed {config['name']}: {e}")
                traceback.print_exc()
    
    
    
def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, result_dir):
    base_dir = f"{result_dir}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    print(f" Saving results to: {base_dir}")
    
    # 1. 결과 요약 저장 
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    # 2. 상세 예측 로그 & 누적 수익률 데이터 생성 
    pred_logs = evaluator.get_prediction_logs()
    
    for model_name, df_log in pred_logs.items():
        # (1) 기본 예측 로그 저장
        df_log.to_csv(f"{base_dir}/predictions_{model_name}.csv")
        
        # (2) [추가] 누적 수익률 곡선 데이터 생성 (Equity Curve)

        if 'actual_return' in df_log.columns: # 만약 수익률 컬럼이 있다면
            df_log['strategy_return'] = df_log['signal'] * df_log['actual_return']
            df_log['cumulative_return'] = (1 + df_log['strategy_return']).cumprod()
            df_log[['cumulative_return']].to_csv(f"{base_dir}/equity_curve_{model_name}.csv")

    # 3. 모델 객체 저장 
    for name, model in evaluator.get_models_dict().items():
        joblib.dump(model, f"{base_dir}/model_{name}.pkl")
            
    # 4. 스케일러 저장 
    if 'scaler' in fold_data:
        joblib.dump(fold_data['scaler'], f"{base_dir}/scaler.pkl")
        
    # 5. [중요] 메타데이터 저장
    feature_names = fold_data.get('selected_features', [])
    if isinstance(feature_names, pd.Index): feature_names = feature_names.tolist()
    
    meta_data = {
        'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'fold_idx': fold_idx,
        'model_thresholds': evaluator.get_best_thresholds(), 
        'input_features': feature_names,                   
        'scaling_needed': True if 'scaler' in fold_data else False,
        'parameters': {
            'profit_mult': fold_data.get('profit_mult'), 
            'stop_mult': fold_data.get('stop_mult'),     
            'lookahead': fold_data.get('lookahead')
        }
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary






In [6]:
def run_optuna_optimization(df_merged, df_hour, ml_models, n_trials=30):
    
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}_Sniper"
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    LOG_PATH = f"{RESULT_DIR}/optuna_logs.csv"

    ohlc_cols = ['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close'] 
    
    lookup_df = df_merged[ohlc_cols].copy()
    lookup_df['date'] = pd.to_datetime(lookup_df['date'])
    lookup_df = lookup_df.set_index('date').sort_index()
    
    # [Resume] 기존 로그 로드
    existing_history = pd.DataFrame()
    if os.path.exists(LOG_PATH):
        try:
            existing_history = pd.read_csv(LOG_PATH)
            print(f"\n[Resume] Loaded {len(existing_history)} existing trials.")
        except: pass

    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w") as f:
            f.write("trial,lookahead,profit_mult,stop_mult,top_n,train_days,score\n")

    def objective(trial):
        nonlocal existing_history 
        
        train_days = trial.suggest_int('train_days', 700, 1000, step=50)
        s_mult = trial.suggest_float('stop_mult', 1.2, 1.8, step=0.1)
        p_mult = trial.suggest_float('profit_mult', 2.0, 3.5, step=0.3)
        lookahead = trial.suggest_int('lookahead', 6, 18, step=2)
        top_n = trial.suggest_int('top_n', 15, 25, step=2)

        # 중복 실행 방지 
        if not existing_history.empty:
            mask = (
                (existing_history['lookahead'] == lookahead) &
                (np.isclose(existing_history['profit_mult'], p_mult, atol=1e-5)) &
                (np.isclose(existing_history['stop_mult'], s_mult, atol=1e-5)) &
                (existing_history['top_n'] == top_n) &
                (existing_history['train_days'] == train_days)
            )
            if mask.any():
                prev_score = existing_history.loc[mask, 'score'].values[0]
                print(f"[Skip] Already done. Score: {prev_score}")
                return prev_score

        trial_name = f"T{trial.number}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}_N{top_n}"
        print(f"\n{'='*60}\n Starting {trial_name}\n{'='*60}")
        
        try:
            # 1. Build Pipeline (데이터 준비)
            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                df_hour=df_hour,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_periods=lookahead,
                profit_mult=p_mult,
                stop_mult=s_mult,
                top_n=top_n,
                initial_train_days=train_days
            )
            
            fold_scores = []
            
            # 2. Walk-Forward Loop
            for fold_data in pipeline_result:
                stats = fold_data.get('stats', {}) 
                fold_idx = stats.get('fold_idx', 0)
                fold_type = stats.get('fold_type', 'unknown')
                
                print(f"   >> Running Fold {fold_idx} ({fold_type})")
                
                fold_data['profit_mult'] = p_mult
                fold_data['stop_mult'] = s_mult
                fold_data['lookahead'] = lookahead 

                test_dates = pd.to_datetime(fold_data['test']['dates'])
                
                test_ohlc = lookup_df.reindex(test_dates).fillna(0)
                
                # =============================================================

                evaluator = ModelEvaluator(save_models=True) 
                trainer = ModelTrainer(evaluator)
                
                # 모델 학습
                trainer.train_all_models(
                    fold_data['train']['X'], fold_data['train']['y'],
                    fold_data['val']['X'], fold_data['val']['y'],
                    fold_data['test']['X'], fold_data['test']['y'],
                    p_mult, s_mult, ml_models,
                    test_dates=test_dates, 
                    test_prices=test_ohlc
                )

                # 결과 및 모델 저장
                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, RESULT_DIR)
                
                # [중요 수정 부분] 점수 집계 (Expectancy)
                # ModelEvaluator.evaluate_model에서 반환하는 키는 "Test_Exp"입니다.
                if 'Test_Exp' in summary.columns:
                    best_fold_score = summary['Test_Exp'].max()
                    fold_scores.append(best_fold_score)
                
                del evaluator, trainer
                gc.collect()
            
            # fold_scores가 비어있으면 -99.0 처리 (여전히 안전장치는 유지)
            final_score = np.mean(fold_scores) if fold_scores else -99.0
            print(f"\n === Trial Score: {final_score:.4f}R ===")
            
            # 로그 기록
            with open(LOG_PATH, "a") as f:
                f.write(f"{trial.number},{lookahead},{p_mult},{s_mult},{top_n},{train_days},{final_score}\n")
                f.flush() 
                os.fsync(f.fileno())
            
            # 메모리 DB 업데이트
            new_row = pd.DataFrame([[trial.number, lookahead, p_mult, s_mult, top_n, train_days, final_score]], 
                                   columns=['trial','lookahead','profit_mult','stop_mult','top_n','train_days','score'])
            if existing_history.empty: existing_history = new_row
            else: existing_history = pd.concat([existing_history, new_row], ignore_index=True)
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial Failed: {e}")
            traceback.print_exc()
            return -99.0

    # Optuna 실행
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print(f"\n[Optuna] Best Params: {study.best_params}")
    return study


In [ ]:
# -----------------------------------------------------------------------------
# 5. Main Execution Block
# -----------------------------------------------------------------------------
df_merged=pd.read_csv("eth_4hour_cal.csv")
df_hour = pd.read_csv("eth_hour.csv")
# [실제 실행]
study = run_optuna_optimization(df_merged, df_hour, ML_MODELS_CLASSIFICATION, n_trials=60)

print("==================================================")
print(f" Best Expectancy: {study.best_value:.4f}")
print("==================================================")



[I 2025-11-29 08:25:23,076] A new study created in memory with name: no-name-6cfc1ea3-686d-4741-bf35-71ef8ebaa43c



[Resume] Loaded 0 existing trials.

 Starting T0_L8_P3.5_S1.4_N21

 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 17818/17901 (Removed: 83)
Final Data Shape: (17818, 221)
Data Split Completed. Total 9 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-04-05 13:00:00 ~ 2022-03-06 09:00:00 (N=4200)
 Val   Period: 2022-03-07 21:00:00 ~ 2022-08-04 17:00:00 (N=900)
 Test  Period: 2022-08-06 05:00:00 ~ 2023-01-03 01:00:00 (N=900)
[Class Balance] Train Set: {0: 0.5311904761904762, 1: 0.4688095238095238}

[Feature Selection] Top 21 Features Selected:
 -> curve-dex_eth_tvl_1d_chg, uniswap_eth_tvl_1d_chg, usdt_eth_mcap_pct_chg_24h, GOLD_pct_chg_24h, ATR_84, MACDS_72_156_54, PRICE_VS_LOW_30p, PRICE_VS_LOW_360p, usdt_eth_mcap_ma180_ratio, DXY_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, makerdao_eth_tvl_ma180_ratio, SP500_ma180_ratio, GOLD_ma180_ratio, SMA_120_ma180_ratio, eth_chain_tvl_pct_chg_24h, makerdao_eth_tvl_1d_chg, curve-dex_eth_tvl_pct_


[Feature Selection] Top 21 Features Selected:
 -> usdt_eth_mcap_pct_chg_24h, lido_eth_tvl_1d_chg, DXY_pct_chg_24h, ATR_84, PRICE_VS_LOW_30p, BREAKOUT_STR_30p, BREAKOUT_STR_120p, DXY_ma180_ratio, VIX_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, SP500_ma180_ratio, GOLD_ma180_ratio, OBV_ma180_ratio, curve-dex_eth_tvl_pct_chg_24h, eth_chain_tvl_1d_chg, makerdao_eth_tvl_1d_chg, eth_chain_tvl_pct_chg_24h, curve-dex_eth_tvl_1d_chg, uniswap_eth_tvl_pct_chg_24h, aave_eth_tvl_pct_chg_24h, optimism_tvl_pct_chg_24h
   >> Running Fold 1 (walk_forward_rolling_reverse)
  [Info] Training with RR_Ratio: 2.50
  [CatBoost]     Train Exp: 2.3229R | Val Exp: 1.8378R (Trades: 37)
    -> [EVAL] CatBoost     | Te_Exp: -0.327R | Te_Trades: 26 | Th: 0.5536
  [RandomForest] Train Exp: 2.3526R | Val Exp: 2.0139R (Trades: 36)
    -> [EVAL] RandomForest | Te_Exp: 0.149R | Te_Trades: 67 | Th: 0.5854
  [LightGBM]     Train Exp: 1.6398R | Val Exp: 1.6047R (Trades: 43)
    -> [EVAL] LightGBM     | Te_Exp: 0.867R | Te_T


[Feature Selection] Top 23 Features Selected:
 -> usdt_eth_mcap_pct_chg_24h, aave_eth_tvl_1d_chg, ATR_84, MACDH_72_156_54, PRICE_VS_LOW_30p, BREAKOUT_STR_30p, usdt_eth_mcap_ma180_ratio, DXY_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, SP500_ma180_ratio, GOLD_ma180_ratio, SMA_120_ma180_ratio, SMA_300_ma180_ratio, eth_chain_tvl_pct_chg_24h, uniswap_eth_tvl_pct_chg_24h, curve-dex_eth_tvl_pct_chg_24h, curve-dex_eth_tvl_1d_chg, eth_chain_tvl_1d_chg, makerdao_eth_tvl_1d_chg, uniswap_eth_tvl_1d_chg, makerdao_eth_tvl_pct_chg_24h, lido_eth_tvl_1d_chg, SP500_pct_chg_24h

 Processing Fold 2 (walk_forward_rolling_reverse)
 Train Period: 2020-09-06 21:00:00 ~ 2022-08-07 17:00:00 (N=4200)
 Val   Period: 2022-08-08 21:00:00 ~ 2023-01-05 17:00:00 (N=900)
 Test  Period: 2023-01-06 21:00:00 ~ 2023-06-05 17:00:00 (N=900)
[Class Balance] Train Set: {0: 0.5095238095238095, 1: 0.49047619047619045}
